# **Track 1 - Discrete Representation**

In [30]:
## IMPORTS
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.neighbors import NearestNeighbors
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import warnings
warnings.filterwarnings('ignore')
from itertools import product
from tqdm import tqdm

In [ ]:
## LOAD THE DATA
dev_df = pd.read_csv('dev_responses.csv')
train_df = pd.read_csv('train_responses.csv')

## REMOVE INVALID RESPONSES
train_df = train_df[
    train_df['model_response'].astype(str).str.strip().replace(r'^\W*$', '', regex=True) != ''
].reset_index(drop=True)

## **TFIDF**

In [27]:
## TFIDF WITH OPTIMAL PARAMS
vectorizer = TfidfVectorizer(
    max_features=20000,  
    ngram_range=(1, 3), 
    sublinear_tf=True,   
    analyzer='char_wb', 
    min_df=1,            
    max_df=0.95         
)

vectorizer.fit(train_df['user_prompt'].tolist())

TfidfVectorizer(analyzer='char_wb', max_df=0.95, max_features=20000,
                ngram_range=(1, 3), sublinear_tf=True)

In [29]:
train_embeddings = vectorizer.transform(train_df['user_prompt']).toarray() 
dev_embeddings = vectorizer.transform(dev_df['user_prompt']).toarray() 

## NORMALIZATION
train_embeddings = normalize(train_embeddings)
dev_embeddings = normalize(dev_embeddings)

In [ ]:
## NEAREST NEIGHBORS
nn = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='cosine')
nn.fit(train_embeddings)

# get nearest neighbor for each dev prompt
distances, indices = nn.kneighbors(dev_embeddings)

In [20]:
results = []
for i, (dev_idx, train_idx) in enumerate(zip(dev_df.index, indices.flatten())):
    results.append({
        'conversation_id': dev_df.loc[dev_idx, 'conversation_id'],
        'user_prompt': dev_df.loc[dev_idx, 'user_prompt'],
        'model_response': str(dev_df.loc[dev_idx, 'model_response']),
        'retrieved_prompt': train_df.loc[train_idx, 'user_prompt'],
        'retrieved_response': str(train_df.loc[train_idx, 'model_response']),
    })

results_df = pd.DataFrame(results)


In [ ]:
## BLEU SCORE CALCULATION
smoothingfunction = SmoothingFunction()

def calculate_bleu(row):
    reference = str(row['model_response']).split()
    candidate = str(row['retrieved_response']).split()
    
    return sentence_bleu(
        [reference], 
        candidate, 
        weights=(0.5, 0.5, 0, 0), 
        smoothing_function=smoothingfunction.method3
    )

results_df['bleu_score'] = results_df.apply(calculate_bleu, axis=1)

In [26]:
print(f"Average BLEU score: {results_df['bleu_score'].mean():.4f}")

Average BLEU score: 0.0885


### **Create Submission CSV**

In [ ]:
def generate_track1_submission(train_df, dev_df, test_df, output_file='track_1_test.csv'):
    combined_df = pd.concat([train_df, dev_df], ignore_index=True)

    combined_df = combined_df[
        combined_df['model_response'].astype(str).str.strip().replace(r'^\W*$', '', regex=True) != ''
    ].reset_index(drop=True)

    vectorizer = TfidfVectorizer(
        max_features=20000,
        ngram_range=(1, 3),
        sublinear_tf=True,
        analyzer='char_wb',
        min_df=1,
        max_df=0.95
    )
    vectorizer.fit(combined_df['user_prompt'].tolist())

    combined_embeddings = vectorizer.transform(combined_df['user_prompt']).toarray()
    test_embeddings = vectorizer.transform(test_df['user_prompt']).toarray()

    combined_embeddings = normalize(combined_embeddings)
    test_embeddings = normalize(test_embeddings)

    nn = NearestNeighbors(n_neighbors=1, algorithm='brute', metric='cosine')
    nn.fit(combined_embeddings)
    _, idxs = nn.kneighbors(test_embeddings)

    matched_ids = [combined_df.iloc[i]['conversation_id'] for i in idxs.flatten()]
    result_df = pd.DataFrame({
        'conversation_id': test_df['conversation_id'],
        'response_id': matched_ids
    })

    result_df.to_csv(output_file, index=False)
    print(f"Saved Track 1 submission to: {output_file}")


In [25]:
test_df = pd.read_csv('test_prompts.csv')
generate_track1_submission(train_df, dev_df, test_df)

Saved Track 1 submission to: track_1_test.csv


### **Hyperparameter Search**

In [13]:
# define hyperparameter grid
param_grid = {
    'ngram_range': [(1, 2), (1, 3)],
    'analyzer': ['char_wb', 'word'],
    'max_features': [10000, 20000],
    'min_df': [1, 2],
    'max_df': [0.9, 0.95],
    'n_neighbors': [1, 3, 5]
}

# cartesian product of all param combinations
param_combinations = list(product(
    param_grid['ngram_range'],
    param_grid['analyzer'],
    param_grid['max_features'],
    param_grid['min_df'],
    param_grid['max_df'],
    param_grid['n_neighbors']
))

best_bleu = -1
best_params = None

for (ngram_range, analyzer, max_features, min_df, max_df, n_neighbors) in tqdm(param_combinations, desc="Grid Search"):
    try:
        vectorizer = TfidfVectorizer(
            ngram_range=ngram_range,
            analyzer=analyzer,
            max_features=max_features,
            min_df=min_df,
            max_df=max_df,
            sublinear_tf=True
        )
        vectorizer.fit(train_df['user_prompt'].tolist())
        train_embeddings = vectorizer.transform(train_df['user_prompt']).toarray()
        dev_embeddings = vectorizer.transform(dev_df['user_prompt']).toarray()
        train_embeddings = normalize(train_embeddings)
        dev_embeddings = normalize(dev_embeddings)

        nn = NearestNeighbors(n_neighbors=n_neighbors, algorithm='brute', metric='cosine')
        nn.fit(train_embeddings)
        _, indices = nn.kneighbors(dev_embeddings)

        retrieved = [train_df.iloc[i]['model_response'] for i in indices[:, 0]]
        bleu_scores = [
            sentence_bleu(
                [str(ref).split()],
                str(hyp).split(),
                weights=(0.5, 0.5, 0, 0),
                smoothing_function=smoothingfunction.method3
            )
            for ref, hyp in zip(dev_df['model_response'], retrieved)
        ]

        avg_bleu = np.mean(bleu_scores)
        if avg_bleu > best_bleu:
            best_bleu = avg_bleu
            best_params = {
                'ngram_range': ngram_range,
                'analyzer': analyzer,
                'max_features': max_features,
                'min_df': min_df,
                'max_df': max_df,
                'n_neighbors': n_neighbors
            }

        print(f"BLEU: {avg_bleu:.4f} | Params: {ngram_range}, {analyzer}, {max_features}, {min_df}, {max_df}, k={n_neighbors}")

    except Exception as e:
        print(f"Skipped: {ngram_range}, {analyzer}, {max_features}, {min_df}, {max_df}, k={n_neighbors} due to error: {e}")

print(f"\nBest BLEU: {best_bleu:.4f} with params: {best_params}")

Grid Search:   1%|          | 1/96 [00:13<20:56, 13.23s/it]

BLEU: 0.0840 | Params: (1, 2), char_wb, 10000, 1, 0.9, k=1


Grid Search:   2%|▏         | 2/96 [00:26<21:08, 13.49s/it]

BLEU: 0.0840 | Params: (1, 2), char_wb, 10000, 1, 0.9, k=3


Grid Search:   3%|▎         | 3/96 [00:40<21:16, 13.73s/it]

...

BLEU: 0.0799 | Params: (1, 3), word, 20000, 2, 0.95, k=1


Grid Search:  99%|█████████▉| 95/96 [46:41<00:50, 50.06s/it]

BLEU: 0.0799 | Params: (1, 3), word, 20000, 2, 0.95, k=3


Grid Search: 100%|██████████| 96/96 [47:41<00:00, 29.80s/it]

BLEU: 0.0799 | Params: (1, 3), word, 20000, 2, 0.95, k=5

Best BLEU: 0.0885 with params: {'ngram_range': (1, 3), 'analyzer': 'char_wb', 'max_features': 20000, 'min_df': 1, 'max_df': 0.95, 'n_neighbors': 1}
